In [1]:
import pandas as pd

In [2]:
ASIGNATURAS = {
    'PROGRAMACION': [2711, 2714],
    'INGLES': [2712],
    'COACH': [2713]
}

In [3]:
MODO_TRIPULANTES_NOREGULARES = False
REVISION = 5 #Es el número que estará de sufijo en el reporte
CICLO = 4
if not MODO_TRIPULANTES_NOREGULARES:
    CONSULTA = pd.read_csv('./INPUT/CHEQUEO/MATRICULAS-C4-29-09.csv', sep=';', encoding='latin-1', skiprows=1)
    REAL = pd.read_excel('./INPUT/CHEQUEO/CONSOLIDADO_MATRICULAS_2022-09-27.xlsx', engine='openpyxl')
else:
    CONSULTA = pd.read_csv('./INPUT/CHEQUEO/MATRICULAS-NR-C3-04-08-22-2.csv', sep=';', encoding='latin-1', skiprows=1)
    REAL = pd.read_excel('./INPUT/CHEQUEO/MATRICULAS_NO_REGULARES_2022-08-04.xlsx', engine='openpyxl')

In [4]:
rta = {
    'CODIGO': [],
    'NOMBRE': [],
    'PROGRAMACION_REAL': [],
    'PROGRAMACION_SA': [],
    'PROGRAMACION_COINCIDE': [],
    'INGLES_REAL': [],
    'INGLES_SA': [],
    'INGLES_COINCIDE': [],
    'COACH_REAL': [],
    'COACH_SA': [],
    'COACH_COINCIDE': [],
}
codigos = CONSULTA['ESTUDIANTE'].unique()
if MODO_TRIPULANTES_NOREGULARES:
    codigosNR = REAL['CODIGO'].unique()
for codigo in codigos:
    if MODO_TRIPULANTES_NOREGULARES and not (codigo in codigosNR):
        continue
    dfEst = CONSULTA[CONSULTA['ESTUDIANTE'] == codigo]
    dfEstReal = REAL[REAL['CODIGO'] == codigo]
    rta['CODIGO'].append(codigo)
    rta['NOMBRE'].append(dfEst['APELLIDOS'].iloc[0] +' '+ dfEst['NOMBRES'].iloc[0])
    for asignatura, codAsignatura in ASIGNATURAS.items():
        grupoReal = dfEstReal[asignatura].iloc[0]
        rta[asignatura + '_REAL'].append(grupoReal)
        temp = []
        for cod in codAsignatura:
            check = dfEst[dfEst['ASIGNATURA'] == cod]
            if len(check) > 0:
                temp.append(check)
#         if len(codAsignatura) > 1:
#             for cod in codAsignatura:
#                 check = dfEst[dfEst['ASIGNATURA'] == cod]
#                 if len(check) > 0:
#                     temp.append(check)
#         else:
#             temp.append(dfEst[dfEst['ASIGNATURA'] == codAsignatura[0]])
        tam = len(temp)
        if tam == 0:
            rta[asignatura + '_SA'].append('NO MATRICULADO')
            rta[asignatura + '_COINCIDE'].append('N/A')
        elif tam == 1:
            grupo = temp[0]['GRUPO'].iloc[0]
            rta[asignatura + '_SA'].append(grupo)
            rta[asignatura + '_COINCIDE'].append(True if grupoReal == grupo else False)
        else:
            grupo = ''
            for df in temp:
                grupo += df['GRUPO'].iloc[0]
            rta[asignatura + '_SA'].append(grupo)
            rta[asignatura + '_COINCIDE'].append(True if grupoReal == grupo else False)
resultados = pd.DataFrame(rta)

In [5]:
from datetime import date
nombre = f'./OUTPUT/CHEQUEO/RevisiónCiclo{str(CICLO)}-{str(date.today())}{"-NR" if MODO_TRIPULANTES_NOREGULARES else ""}-{REVISION}.xlsx'
resultados.to_excel(nombre, index = False)